
# 📘 Career Assistant – ChatGPT-Powered (Gradio UI + Local CV & Background Files)

This notebook loads:

- **CV** → from a local `cv.pdf`
- **Background Information** → from a local `background.txt`

Then it automatically builds a **structured career profile** using ChatGPT.

The Gradio UI only handles:
- Recruiter's question → Assistant answers as the candidate


In [ ]:

!pip install --quiet openai gradio pypdf


In [1]:

import os
from pypdf import PdfReader
from openai import OpenAI
import gradio as gr


from dotenv import load_dotenv
from agents import Agent, Runner, trace
# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


c:\PROJECTS\career assistant\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'agents'

## 📄 Load CV (PDF) and Background Text

In [ ]:

def extract_pdf_text(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        txt = page.extract_text()
        if txt:
            text += txt + "\n"
    return text.strip()

# Load files
CV_PATH = "cv.pdf"
BACKGROUND_PATH = "background.txt"

cv_text = extract_pdf_text(CV_PATH)

with open(BACKGROUND_PATH, "r", encoding="utf-8") as f:
    background_text = f.read().strip()

cv_text[:500], background_text[:300]  # preview


## 🧠 Build Structured Career Profile Automatically

In [ ]:

def build_career_profile(cv_text, background):
    prompt = f"""
Extract a structured career profile based strictly on the information provided.

CV CONTENT:
{cv_text}

BACKGROUND INFORMATION:
{background}

Return a structured professional profile containing:
- Key skills
- Technical tools
- Work experience highlights
- Academic strengths
- Certifications (if any)
- Major projects & impact
- Soft skills
- Career preferences
- Strengths relevant to recruiters
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You extract structured candidate profiles."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

career_profile = build_career_profile(cv_text, background_text)
career_profile[:800]  # preview


## 🎙️ Recruiter Question → Candidate Answers

In [ ]:

def answer_question(profile, question):
    prompt = f"""
You are acting as the candidate described below.
Use only the information in the profile.
If unsure, give a safe, reasonable answer.

PROFILE:
{profile}

RECRUITER QUESTION:
{question}

Answer in first person ("I") and professionally.
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You answer interview questions as the candidate."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()


## 🖥️ Gradio UI – Recruiter Q&A Interface

In [ ]:

with gr.Blocks() as demo:
    gr.Markdown("# 🤖 Career Assistant – ChatGPT (CV & Background Loaded From Code)")
    gr.Markdown("Ask any recruiter question. The assistant will answer as the candidate.")
    
    question = gr.Textbox(label="Recruiter's Question", lines=2)
    answer = gr.Textbox(label="Candidate's Response", lines=8)
    
    ask_btn = gr.Button("Ask")
    
    ask_btn.click(
        fn=answer_question,
        inputs=[gr.State(career_profile), question],
        outputs=answer
    )

demo.launch()


### With Guardrails

In [ ]:
### 🔒 Demonstrating OpenAI’s Built-In Guardrails (Natural Language Mode)

from openai import OpenAI
client = OpenAI()

def guarded_interview_response(question):
    """
    Demonstrates OpenAI’s built-in guardrails:
    - Detects inappropriate / discriminatory interview questions
    - Automatically refuses unsafe content
    - Keeps responses professional and safe
    - Uses OpenAI's native safety system (not manual rules)
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a job interview candidate with OpenAI safety guardrails enabled.\n"
                    "Follow these rules strictly:\n"
                    "1. Reject inappropriate, discriminatory, or illegal interview questions.\n"
                    "2. If a question violates employment law or privacy norms, politely decline.\n"
                    "3. If the question is allowed, answer normally and professionally.\n"
                    "4. NEVER reveal or fabricate sensitive attributes (age, race, religion, politics, etc.).\n"
                    "5. Keep responses natural and conversational — do NOT use JSON or structured formats.\n"
                    "6. Use refusal guidelines when necessary:\n"
                    "   - “I'm not able to answer that question.”\n"
                    "   - “That’s not appropriate for a job interview.”\n"
                )
            },
            {"role": "user", "content": question}
        ],
        # built-in guardrails feature (OpenAI SDK, 2024+)
        response_format={"type": "guardrails"}  
    )

    return response.choices[0].message.content


# 🔍 Try guardrails demo
test_questions = [
    "Do you plan on having children soon?",
    "What's your religion?",
    "What is your greatest strength?",
    "Tell me about your experience working with Python.",
]

for q in test_questions:
    print(f"Q: {q}")
    print("A:", guarded_interview_response(q))
    print("-" * 60)
